## 使用Gensim Word2Vec模型訓練維基數據

### 1.维基百科-條目文本整合下載
 - 先下載維基百科條目文本
https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2
 - 建立Word2Vec 資料夾，待會用到的檔案都放這邊

### 2.安裝套件
 - pip install gensim 
 - 確定你的Python 有裝 cython 沒有的話就 pip install cython

### 3.數據提取
 - 首先需要從壓縮包中提取出中文維基的條目文本。在Word2Vec路徑下建立process_wiki.py文件
 - cmd 切換到Word2Vec路徑 指令碼 python process_wiki.py zhwiki-latest-pages-articles.xml.bz2 wiki.zh.text 
 - 執行完會多個 wiki.zh.text
 - process_wiki.py 如下:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
if __name__ == '__main__':
    program = os.path.basename(sys.argv[0])
    logger = logging.getLogger(program)
 
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info("running %s" % ' '.join(sys.argv))
 
    # check and process input arguments
    if len(sys.argv) < 3:
        print (globals()['__doc__'] % locals())
        sys.exit(1)
    inp, outp = sys.argv[1:3]
    space = " "
    i = 0
 
    output = open(outp, 'w')
    wiki = WikiCorpus(inp, lemmatize=False, dictionary={})
    for text in wiki.get_texts():
        output.write(space.join(text) + "\n")
        i = i + 1
        if (i % 10000 == 0):
            logger.info("Saved " + str(i) + " articles")
 
    output.close()
    logger.info("Finished Saved " + str(i) + " articles")

### 4.使用結巴進行中文分詞
 - 建議使用繁體字典，跑出來的結果會比較偏向常用詞
 - 這邊大概要跑1-2小時，看資料量。

In [ ]:
import jieba
import sys 
reload(sys)
sys.setdefaultencoding('utf8')
jieba.set_dictionary('dict.txt.big')
jieba.load_userdict("dict_keyw.txt")     
jieba.load_userdict("dict_cbdic.txt")

 
fr = open('wiki.zh.text', 'r')
fw = open('wiki.zh.word.text', 'w')
 
count = 0
for line in fr:
    count += 1
    print count 
 
    result = ''
    line = line.rstrip('\n').split()
    for item in line:
        item = jieba.cut(item)
        for i in item:
            result += i + ' '
    fw.write(result[:-1] + '\n')
 
fr.close()
fw.close()

###  5.模型訓練
 - 使用gensim包提供的word2vec模型進行訓練
 - 指令碼：python train_word2vec_model.py wiki.zh.word.text wiki.zh.text.model wiki.zh.text.vector 
 - 訓練完畢會有四個檔 wiki.zh.text.model、wiki.zh.text.model.syn0.npy、wiki.zh.text.model.syn1.npy、wiki.zh.text.vecto
 - 模型訓練請用python3執行，剛開始使用py2速度慢到會讓你想死。
 - train_word2vec_model.py如下：

In [ ]:
import logging
import os.path
import sys
import multiprocessing
 
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
if __name__ == '__main__':
    program = os.path.basename(sys.argv[0])
    logger = logging.getLogger(program)
 
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info("running %s" % ' '.join(sys.argv))
 
    # check and process input arguments
    if len(sys.argv) < 4:
        print (globals()['__doc__'] % locals())
        sys.exit(1)
    inp, outp1, outp2 = sys.argv[1:4]
 
    model = Word2Vec(LineSentence(inp), size=400, window=5, min_count=5,
            workers=multiprocessing.cpu_count())
 
    # trim unneeded model memory = use(much) less RAM
    #model.init_sims(replace=True)
    model.save(outp1)
    model.save_word2vec_format(outp2, binary=False)

### 6.模型使用
 - 上面四個檔要放一起
 - most_similar() 給定詞語遍歷其他所有詞語並返回與其相關度最高的10個詞語
 - similarity() 返回兩個詞語的相關度

In [2]:
import gensim
model = gensim.models.Word2Vec.load("wiki.zh.text.model")

In [6]:
result = model.most_similar('足球')
for ele in result:
    print(ele[0],ele[1])

足球运动 0.5966246724128723
冰球 0.5476202368736267
排球 0.5415818691253662
手球 0.527426540851593
板球 0.5252264738082886
美式足球 0.5022620558738708
英超球 0.49216344952583313
踢球 0.49188557267189026
世界足球 0.48957183957099915
棒球 0.48686718940734863


In [7]:
print(model.similarity('男人','女人'))

0.757236245216


In [8]:
result = model.most_similar('青蛙')
for ele in result:
    print(ele[0],ele[1])

蟑螂 0.6209989190101624
烏龜 0.6205072402954102
螃蟹 0.612657904624939
老鼠 0.5995995402336121
蚱蜢 0.5932121276855469
小狗 0.5863862037658691
蜥蜴 0.585473895072937
猩猩 0.5825787782669067
巫婆 0.5785835981369019
狐狸 0.5750727653503418


In [9]:
result = model.most_similar('女人')
for ele in result:
    print(ele[0],ele[1])

男人 0.757236123085022
家伙 0.520851731300354
傻瓜 0.514291524887085
陌生人 0.49404311180114746
老公 0.4916646182537079
女孩 0.48187896609306335
女孩子 0.4746483862400055
老婆 0.46345457434654236
女明星 0.4593360424041748
撒嬌 0.45863184332847595


In [10]:
result = model.most_similar('衣服')
for ele in result:
    print(ele[0],ele[1])

鞋子 0.7607700228691101
衣物 0.7605881094932556
裙子 0.717649519443512
大衣 0.6882219910621643
外衣 0.6878334283828735
外套 0.6821765899658203
內褲 0.6808929443359375
褲子 0.6739594340324402
上衣 0.6662149429321289
背心 0.6620113849639893


### 參考資料  
#### http://zhanghonglun.cn/blog/
#### http://www.52nlp.cn/
